In [ ]:
import os
import tempfile as tmp
import warnings

import sklearn.model_selection

# os.environ user environment variable
os.environ['JOBLIB_TEMP_FOLDER'] = tmp.gettempdir() # directory
os.environ['OMP_NUM_THREADS'] = '1' 
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from autoPyTorch.api.tabular_classification import TabularClassificationTask

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [ ]:
# import pandas as pd

# X = pd.read_csv('Datasets/CICDDoS2019/feature.csv')
# y = pd.read_csv('Datasets/CICDDoS2019/label.csv')

# X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
#     X,
#     y,
#     random_state=1,
# )

import sklearn.datasets

X, y = sklearn.datasets.fetch_openml(data_id=40981, return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X,
    y,
    random_state=1,
)

In [ ]:
from autoPyTorch.datasets.resampling_strategy import CrossValTypes, HoldoutValTypes

api = TabularClassificationTask(
    temporary_directory='./Test_Results/CDCNNBackbone_test/autoPyTorch_example_tmp',
    output_directory='./Test_Results/CDCNNBackbone_test/autoPyTorch_example_out',
    include_components = {'imputer': 'None', 'network_backbone': ['CDCNNBackbone'],
                          'network_init': ["KaimingInit", "NoInit", "XavierInit"]},
    ensemble_size = 0, # ensembling disable
    resampling_strategy=HoldoutValTypes.holdout_validation,
    resampling_strategy_args={'val_share': 0.2}
)

In [ ]:
import numpy as np

api.search(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test.copy(),
    y_test=y_test.copy(),
    optimize_metric='accuracy',
#     memory_limit=1000000,
    total_walltime_limit=3600,
    func_eval_time_limit_secs=3600,
    enable_traditional_pipeline=False,
)

[WARNING] [2023-04-02 17:14:16,802:Client-AutoPyTorch:bac7e613-d136-11ed-bf0a-0242ac110002:1] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (3596.000000)
[WARNING] [2023-04-02 18:14:22,134:Client-autoPyTorch.automl_common.common.utils.backend] Directory ./Test_Results/CDCNNBackbone_3600/autoPyTorch_example_tmp/.autoPyTorch/ensembles does not exist
[ERROR] [2023-04-02 18:14:22,137:Client-AutoPyTorch:bac7e613-d136-11ed-bf0a-0242ac110002:1] No valid ensemble was created. Please check the logfile for errors. Default to the best individual estimator:[(1, 2, 5.555555555555555)]
NoneType: None


In [ ]:
y_pred = api.predict(X_test)
score = api.score(y_pred, y_test)
print(score)

# Print statistics from search
print(api.sprint_statistics())
print(api.show_models())

{'accuracy': 0.7630057803468208}
autoPyTorch results:
	Dataset name: bac7e613-d136-11ed-bf0a-0242ac110002
	Optimisation Metric: accuracy
	Best validation score: 0.8173076923076923
	Number of target algorithm runs: 14
	Number of successful target algorithm runs: 13
	Number of crashed target algorithm runs: 0
	Number of target algorithms that exceeded the time limit: 1
	Number of target algorithms that exceeded the memory limit: 0

|    | Preprocessing                                                                                    | Estimator                                                   |   Weight |
|---:|:-------------------------------------------------------------------------------------------------|:------------------------------------------------------------|---------:|
|  0 | SimpleImputer,Variance Threshold,NoCoalescer,OneHotEncoder,StandardScaler,NoFeaturePreprocessing | no embedding,CDCNNBackbone,FullyConnectedHead,nn.Sequential |        1 |


In [ ]:
api.refit(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    memory_limit=1000000,
    total_walltime_limit=600,
    run_time_limit_secs=300,
    # you can change the resampling strategy to
    # for example, CrossValTypes.k_fold_cross_validation
    # to fit k fold models and have a voting classifier
    # resampling_strategy=CrossValTypes.k_fold_cross_validation
)

In [ ]:
y_pred = api.predict(X_test)
score = api.score(y_pred, y_test)
print(score)

# Print the final ensemble built by AutoPyTorch
print(api.show_models())